In [1]:
import pandas as pd
import requests
import bs4
import re
from neo4j import GraphDatabase

In [2]:
#item
data = requests.get('https://app.mobalytics.gg/lol/champions/aatrox/build')
soup = bs4.BeautifulSoup(data.text)
item = soup.find('div',{'class':'m-1y0hiws'})
name = item.find_all('img')
name_list = []
for i in name : 
    nl = i ['alt']
    name_list.append(nl)
name_list[-6:]

AttributeError: 'NoneType' object has no attribute 'find_all'

In [5]:
#champion
src1 = requests.get('https://app.mobalytics.gg/lol/champions')
soup1 = bs4.BeautifulSoup(src1.text, 'lxml')
li = soup1.find('div', {'class':'m-1u0kasc'})
ChampName = li.find_all('div', {'class':'m-15ivlsc'})
ChampName_list = []
for i in ChampName:
    ChampName_list.append(i.text)
ChampName_list

AttributeError: 'NoneType' object has no attribute 'find'

In [12]:
#link champion
a_tags = li.find_all('a')
link_rows = []
url_prefix = 'https://app.mobalytics.gg/'
for i in a_tags:
    link = url_prefix + i['href']
    link_rows.append(link)
link_rows

['https://app.mobalytics.gg//lol/champions/aatrox/build',
 'https://app.mobalytics.gg//lol/champions/ahri/build',
 'https://app.mobalytics.gg//lol/champions/akali/build',
 'https://app.mobalytics.gg//lol/champions/akshan/build',
 'https://app.mobalytics.gg//lol/champions/alistar/build',
 'https://app.mobalytics.gg//lol/champions/amumu/build',
 'https://app.mobalytics.gg//lol/champions/anivia/build',
 'https://app.mobalytics.gg//lol/champions/annie/build',
 'https://app.mobalytics.gg//lol/champions/aphelios/build',
 'https://app.mobalytics.gg//lol/champions/ashe/build',
 'https://app.mobalytics.gg//lol/champions/aurelionsol/build',
 'https://app.mobalytics.gg//lol/champions/azir/build',
 'https://app.mobalytics.gg//lol/champions/bard/build',
 'https://app.mobalytics.gg//lol/champions/belveth/build',
 'https://app.mobalytics.gg//lol/champions/blitzcrank/build',
 'https://app.mobalytics.gg//lol/champions/brand/build',
 'https://app.mobalytics.gg//lol/champions/braum/build',
 'https://app.

In [13]:
item_list = []

for i in link_rows:
    data1 = requests.get(i)
    soup2 = bs4.BeautifulSoup(data1.text)
    item_champions = soup2.find('div' , {'class':'m-1y0hiws'})
    name_item_chap = item_champions.find_all('img')
    
    name_list = []
    for i in name_item_chap : 
        nl = i ['alt']
        name_list.append(nl)
    
    item_list.append(name_list[-6:])
    
table = pd.DataFrame([ChampName_list, item_list]).transpose()
table

,0,1
0,Aatrox,"[Eclipse, Plated Steelcaps, Death's Dance, Bla..."
1,Ahri,"[Everfrost, Sorcerer's Shoes, Shadowflame, Zho..."
2,Akali,"[Hextech Rocketbelt, Sorcerer's Shoes, Shadowf..."
3,Akshan,"[Kraken Slayer, Berserker's Greaves, Blade of ..."
4,Alistar,"[Bulwark of the Mountain, Mobility Boots, Lock..."
...,...,...
157,Zeri,"[Immortal Shieldbow, Berserker's Greaves, Phan..."
158,Ziggs,"[Liandry's Anguish, Sorcerer's Shoes, Archange..."
159,Zilean,"[Shard of True Ice, Ionian Boots of Lucidity, ..."
160,Zoe,"[Luden's Tempest, Sorcerer's Shoes, Shadowflam..."


In [14]:
first_item_list = []
second_item_list = []
third_item_list = []
fourth_item_list = []
fifth_item_list = []
sixth_item_list = []
for i in item_list:
    first_item_list.append(i[0])
    second_item_list.append(i[1])
    third_item_list.append(i[2])
    fourth_item_list.append(i[3])
    fifth_item_list.append(i[4])
    sixth_item_list.append(i[5])

In [15]:
TableForReccommentItem = pd.DataFrame([ChampName_list, first_item_list, second_item_list, third_item_list, fourth_item_list, fifth_item_list, sixth_item_list]).transpose()
use_item_list = []
for i in item_list:
    for e in i:
        use_item_list.append(e)
TableForItem = pd.DataFrame([set(use_item_list)]).transpose()

In [16]:
#to create node
from requests.sessions import session
neo_data_item = TableForItem.values.tolist()
neo_data_recomment_item = TableForReccommentItem.values.tolist()
neo_exec_comm = []

for i in neo_data_recomment_item:
  neo4j_create_statement_recomment_item = "create (Ritem:RecommentItem {ForChampName :"+ repr(i[0]) +", First :"+ repr(i[1]) +", Second :"+ repr(i[2]) +", Third :"+ repr(i[3]) +", Fourth :"+ repr(i[4]) +", Fifth :"+ repr(i[5]) +", Sixth :"+ repr(i[6]) +"})"
  neo_exec_comm.append(neo4j_create_statement_recomment_item)
for i in neo_data_item:
  neo4j_create_statement_item = "create (item:Item {ItemName :"+ repr(i[0]) +"})"
  neo_exec_comm.append(neo4j_create_statement_item)

def exec_neo_data(neo_exec_comm):
  data_base_connect = GraphDatabase.driver(uri = "bolt://localhost:7687", auth = ("neo4j", "test1"))#target on 'project' db, pls change destination
  session = data_base_connect.session()
  for i in neo_exec_comm:
    session.run(i)

exec_neo_data(neo_exec_comm)

In [37]:
TableForReccommentItem

,0,1,2,3,4,5,6
0,Aatrox,Goredrinker,Plated Steelcaps,Death's Dance,Serylda's Grudge,Maw of Malmortius,Sterak's Gage
1,Ahri,Everfrost,Sorcerer's Shoes,Shadowflame,Zhonya's Hourglass,Rabadon's Deathcap,Void Staff
2,Akali,Hextech Rocketbelt,Sorcerer's Shoes,Shadowflame,Zhonya's Hourglass,Rabadon's Deathcap,Void Staff
3,Akshan,Kraken Slayer,Berserker's Greaves,Blade of The Ruined King,Lord Dominik's Regards,Guinsoo's Rageblade,Wit's End
4,Alistar,Bulwark of the Mountain,Mobility Boots,Locket of the Iron Solari,Thornmail,Zeke's Convergence,Knight's Vow
...,...,...,...,...,...,...,...
156,Zeri,Immortal Shieldbow,Berserker's Greaves,Phantom Dancer,Infinity Edge,Lord Dominik's Regards,Guardian Angel
157,Ziggs,Liandry's Anguish,Sorcerer's Shoes,Archangel's Staff,Shadowflame,Zhonya's Hourglass,Rabadon's Deathcap
158,Zilean,Shard of True Ice,Ionian Boots of Lucidity,Shurelya's Battlesong,Chemtech Putrifier,Zhonya's Hourglass,Archangel's Staff
159,Zoe,Luden's Tempest,Sorcerer's Shoes,Shadowflame,Zhonya's Hourglass,Rabadon's Deathcap,Void Staff


In [31]:
TableForItem

,0
0,Archangel's Staff
1,Immortal Shieldbow
2,Rabadon's Deathcap
3,Knight's Vow
4,Eclipse
...,...
87,Essence Reaver
88,Rylai's Crystal Scepter
89,Umbral Glaive
90,Turbo Chemtank


[['Aatrox',
  'Goredrinker',
  'Plated Steelcaps',
  "Death's Dance",
  "Serylda's Grudge",
  'Maw of Malmortius',
  "Sterak's Gage"],
 ['Ahri',
  'Everfrost',
  "Sorcerer's Shoes",
  'Shadowflame',
  "Zhonya's Hourglass",
  "Rabadon's Deathcap",
  'Void Staff'],
 ['Akali',
  'Hextech Rocketbelt',
  "Sorcerer's Shoes",
  'Shadowflame',
  "Zhonya's Hourglass",
  "Rabadon's Deathcap",
  'Void Staff'],
 ['Akshan',
  'Kraken Slayer',
  "Berserker's Greaves",
  'Blade of The Ruined King',
  "Lord Dominik's Regards",
  "Guinsoo's Rageblade",
  "Wit's End"],
 ['Alistar',
  'Bulwark of the Mountain',
  'Mobility Boots',
  'Locket of the Iron Solari',
  'Thornmail',
  "Zeke's Convergence",
  "Knight's Vow"],
 ['Amumu',
  'Sunfire Aegis',
  'Plated Steelcaps',
  'Demonic Embrace',
  'Thornmail',
  'Force of Nature',
  'Frozen Heart'],
 ['Anivia',
  'Everfrost',
  "Sorcerer's Shoes",
  "Archangel's Staff",
  "Zhonya's Hourglass",
  'Shadowflame',
  "Rabadon's Deathcap"],
 ['Annie',
  "Luden's Tem